In [8]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Reading the CSV file
df = pd.read_csv('../data/Churn_Modelling.csv')

# Display the first few rows of the dataset
df.head()


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42.0,2,0.00,1,1.0,1.0,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41.0,1,83807.86,1,0.0,1.0,112542.58,0
2,3,15619304,Onio,502,France,Female,42.0,8,159660.80,3,1.0,0.0,113931.57,1
3,4,15701354,Boni,699,France,Female,39.0,1,0.00,2,0.0,0.0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43.0,2,125510.82,1,NaN,1.0,79084.10,0


In [4]:
# Checking for missing values
df.isnull().sum()


RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [9]:
# Check the types of each column
print(df.dtypes)


RowNumber            int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                float64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard          float64
IsActiveMember     float64
EstimatedSalary    float64
Exited               int64
dtype: object


In [3]:
df = df.dropna()


In [10]:
# Drop the Surname column
df = df.drop('Surname', axis=1)


In [11]:
# One-Hot Encode the categorical columns
df_encoded = pd.get_dummies(df, columns=['Geography', 'Gender'], drop_first=True)


In [12]:

X = df_encoded.drop('Exited', axis=1)
y = df_encoded['Exited']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import resample
from sklearn.metrics import accuracy_score
import numpy as np

class RandomForestFromScratch:
    def __init__(self, n_trees=100, max_depth=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            # Bootstrap sampling
            X_sample, y_sample = resample(X, y)
            
            # Train a decision tree
            tree = DecisionTreeClassifier(max_depth=self.max_depth)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        # Get predictions from all trees
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        
        # Majority vote
        majority_vote = [np.bincount(tree_preds[:, i]).argmax() for i in range(X.shape[0])]
        return np.array(majority_vote)


In [14]:
# Initialize and train the Random Forest
rf = RandomForestFromScratch(n_trees=100, max_depth=10)
rf.fit(X_train, y_train)


In [15]:
# Make predictions on the test set
y_pred = rf.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')


Accuracy: 0.86


In [16]:
from sklearn.metrics import confusion_matrix, classification_report

y_pred = rf.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Confusion Matrix:
 [[1532   67]
 [ 213  189]]

Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.96      0.92      1599
           1       0.74      0.47      0.57       402

    accuracy                           0.86      2001
   macro avg       0.81      0.71      0.75      2001
weighted avg       0.85      0.86      0.85      2001



In [17]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(rf, X, y, cv=5)
print(f"Cross-Validation Scores: {cv_scores}")
print(f"Mean CV Score: {cv_scores.mean():.2f}")


TypeError: If no scoring is specified, the estimator passed should have a 'score' method. The estimator <__main__.RandomForestFromScratch object at 0x000001BDBB178740> does not.